In [ ]:
# =============================================================================
# 전용면적 이상치 클리핑 모듈 (area_clipping_module.ipynb)
# 작성자: [김병현]
# 생성일: 2025-07-16
# 설명: 부동산 데이터의 전용면적 이상치를 클리핑하여 제거하는 전처리 모듈
# =============================================================================

def clip_area_outliers(df, area_col='전용면적', percentile_low=0.5, percentile_high=99.5, 
                      new_col_name=None, visualize=True, verbose=True):
    """
    전용면적 이상치 클리핑 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        처리할 데이터프레임 (concat 데이터)
    area_col : str, default='전용면적'
        클리핑할 면적 컬럼명
    percentile_low : float, default=0.5
        하위 백분위수 경계값 (0.5% = 하위 0.5%)
    percentile_high : float, default=99.5
        상위 백분위수 경계값 (99.5% = 상위 99.5%)
    new_col_name : str, optional
        새로운 컬럼명. None이면 '{area_col}_clip' 형태로 자동 생성
    visualize : bool, default=True
        클리핑 전후 분포 시각화 여부
    verbose : bool, default=True
        클리핑 정보 출력 여부
        
    Returns:
    --------
    pandas.DataFrame
        클리핑된 컬럼이 추가된 데이터프레임
    dict
        클리핑 정보 (경계값, 제거된 데이터 수 등)
    """
    
    # 새 컬럼명 설정
    if new_col_name is None:
        new_col_name = f'{area_col}_clip'
    
    # 원본 데이터 복사 (원본 데이터를 보존하기 위해 copy() 사용)
    df[new_col_name] = df[area_col].copy()
    
    # 백분위수를 소수점으로 변환 (예: 0.5 -> 0.005, 99.5 -> 0.995)
    q_low = df[new_col_name].quantile(percentile_low / 100)
    q_high = df[new_col_name].quantile(percentile_high / 100)
    
    # 클리핑 전 데이터 개수 확인
    total_count = len(df[new_col_name].dropna())  # 결측치 제외한 전체 개수
    before_clip_count = len(df[(df[new_col_name] < q_low) | (df[new_col_name] > q_high)].dropna())
    
    # 클리핑 적용 (pandas의 clip 함수: 지정된 범위를 벗어나는 값을 경계값으로 제한)
    df[new_col_name] = df[new_col_name].clip(lower=q_low, upper=q_high)
    
    # 클리핑 정보 저장
    clip_info = {
        'lower_bound': q_low,
        'upper_bound': q_high,
        'total_count': total_count,
        'clipped_count': before_clip_count,
        'clipped_ratio': before_clip_count / total_count * 100 if total_count > 0 else 0
    }
    
    # 결과 출력
    if verbose:
        print(f"[{area_col}] 클리핑 완료!")
        print(f"   총 데이터: {total_count:,}개")
        print(f"   클리핑된 데이터: {before_clip_count:,}개 ({clip_info['clipped_ratio']:.2f}%)")
        print(f"   클리핑 경계:")
        print(f"      - 하위 {percentile_low}%: {q_low:.2f}㎡")
        print(f"      - 상위 {percentile_high}%: {q_high:.2f}㎡")
        print(f"   새 컬럼명: '{new_col_name}'")
    
    # 시각화
    if visualize:
        plt.figure(figsize=(15, 5))
        
        # 클리핑 전 분포
        plt.subplot(1, 3, 1)
        sns.histplot(df[area_col].dropna(), bins=100, kde=True, color='lightcoral', alpha=0.7)
        plt.axvline(q_low, color='red', linestyle='--', alpha=0.8, label=f'하위 {percentile_low}%: {q_low:.1f}㎡')
        plt.axvline(q_high, color='red', linestyle='--', alpha=0.8, label=f'상위 {percentile_high}%: {q_high:.1f}㎡')
        plt.title(f'{area_col} (클리핑 전)')
        plt.xlabel('전용면적(㎡)')
        plt.ylabel('빈도')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 클리핑 후 분포
        plt.subplot(1, 3, 2)
        sns.histplot(df[new_col_name].dropna(), bins=100, kde=True, color='cornflowerblue', alpha=0.7)
        plt.title(f'{new_col_name} (클리핑 후)')
        plt.xlabel('전용면적(㎡)')
        plt.ylabel('빈도')
        plt.grid(True, alpha=0.3)
        
        # 클리핑 전후 박스플롯 비교
        plt.subplot(1, 3, 3)
        data_for_boxplot = [df[area_col].dropna(), df[new_col_name].dropna()]
        plt.boxplot(data_for_boxplot, labels=['클리핑 전', '클리핑 후'], patch_artist=True,
                   boxprops=dict(facecolor='lightblue', alpha=0.7))
        plt.title('클리핑 전후 비교')
        plt.ylabel('전용면적(㎡)')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return df, clip_info


def apply_area_clipping(concat):
    """
    베이스 코드에서 호출할 전용면적 클리핑 적용 함수
    
    Parameters:
    -----------
    concat : pandas.DataFrame
        train과 test가 합쳐진 데이터프레임
        
    Returns:
    --------
    pandas.DataFrame
        클리핑이 적용된 데이터프레임
    """
    
    print("\n" + "="*60)
    print("전용면적 이상치 클리핑 시작")
    print("="*60)
    
    # 클리핑 적용
    concat, clip_info = clip_area_outliers(
        df=concat,
        area_col='전용면적',  # 베이스 코드에서는 컬럼명이 '전용면적'으로 변경됨
        percentile_low=0.5,   # 하위 0.5% 제거
        percentile_high=99.5, # 상위 0.5% 제거 (상위 99.5% 유지)
        new_col_name='전용면적_clip',
        visualize=True,
        verbose=True
    )
    
    # 기존 전용면적 컬럼을 클리핑된 값으로 대체 (선택사항)
    # concat['전용면적'] = concat['전용면적_clip']
    # concat.drop('전용면적_clip', axis=1, inplace=True)
    
    print("전용면적 클리핑 완료!")
    print("="*60)
    
    return concat

print("전용면적 클리핑 모듈 로드 완료!")
print("사용법: %run area_clipping_module.ipynb")
print("       concat = apply_area_clipping(concat)")